#### modify M76_valuation_INT to include continuous dividend rate

In [1]:
#
# Valuation of European Call Options
# in Merton's (1976) Jump Diffusion Model
# via Numerical Integration
# 08_m76/M76_valuation_INT.py
#
# (c) Dr. Yves J. Hilpisch
# Derivatives Analytics with Python
#
import math
import numpy as np
from scipy.integrate import quad

#
# Model Parameters
#
S0 = 100.0  # initial index level
K = 100.0  # strike level
T = 1.0  # call option maturity
r = 0.05  # constant short rate
d = 0.05  # dividend yield
sigma = 0.4  # constant volatility of diffusion
lamb = 1.0  # jump frequency p.a.
mu = -0.2  # expected jump size
delta = 0.1  # jump size volatility

#
# Valuation by Integration
#


In [2]:


def M76_value_call_INT(S0, K, T, r, d, sigma, lamb, mu, delta):
    ''' Valuation of European call option in M76 model via
    Lewis (2001) Fourier-based approach.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    sigma: float
        volatility factor in diffusion term
    lamb: float
        jump intensity
    mu: float
        expected jump size
    delta: float
        standard deviation of jump

    Returns
    =======
    call_value: float
        European call option present value
    '''
    int_value = quad(lambda u: M76_integration_function(u, S0, K, T, r, d,
                    sigma, lamb, mu, delta), 0, 50, limit=250)[0]
    call_value = S0 - np.exp(-(r - d) * T) * math.sqrt(S0 * K) / math.pi * int_value
    return call_value


def M76_integration_function(u, S0, K, T, r, d, sigma, lamb, mu, delta):
    ''' Valuation of European call option in M76 model via
    Lewis (2001) Fourier-based approach: integration function.

    Parameter definitions see function M76_value_call_INT. '''
    JDCF = M76_characteristic_function(u - 0.5 * 1j, T, r, d,
                                       sigma, lamb, mu, delta)
    value = 1 / (u ** 2 + 0.25) * (np.exp(1j * u * math.log(S0 / K)) *
                                   JDCF).real
    return value


def M76_characteristic_function(u, T, r, d, sigma, lamb, mu, delta):
    ''' Valuation of European call option in M76 model via
    Lewis (2001) Fourier-based approach: characteristic function.

    Parameter definitions see function M76_value_call_INT. '''
    omega = (r - d) - 0.5 * sigma ** 2 - lamb * (np.exp(mu + 0.5 * delta ** 2) - 1)
    value = np.exp((1j * u * omega - 0.5 * u ** 2 * sigma ** 2 +
            lamb * (np.exp(1j * u * mu - u ** 2 * delta ** 2 * 0.5) - 1)) * T)
    return value



#### modify M76_valuation_INT so Levy process is the variance gamma process

In [3]:

#
# Valuation by Integration
#


def M76_value_call_INT(S0, K, T, r, sigma, lamb, mu, delta):
    ''' Valuation of European call option in M76 model via
    Lewis (2001) Fourier-based approach.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    sigma: float
        volatility factor in diffusion term
    lamb: float
        jump intensity
    mu: float
        expected jump size
    delta: float
        standard deviation of jump

    Returns
    =======
    call_value: float
        European call option present value
    '''
    int_value = quad(lambda u: M76_integration_function(u, S0, K, T, r,
                     sigma, lamb, mu, delta), 0, 50, limit=250)[0]
    call_value = S0 - np.exp(-r * T) * math.sqrt(S0 * K) / math.pi * int_value
    return call_value


def M76_integration_function(u, S0, K, T, r, sigma, lamb, mu, delta):
    ''' Valuation of European call option in M76 model via
    Lewis (2001) Fourier-based approach: integration function.

    Parameter definitions see function M76_value_call_INT. '''
    JDCF = M76_characteristic_function(u - 0.5 * 1j, T, r,
                                       sigma, lamb, mu, delta)

    print ('strip regularity has been broken, valuation is false') if JDCF == 0 else None

    value = 1 / (u ** 2 + 0.25) * (np.exp(1j * u * math.log(S0 / K)) *
                                   JDCF).real
    return value


# modify this so that the Levy process is the variance gamma process

# def M76_characteristic_function(u, T, r, sigma, lamb, mu, delta):
#     ''' Valuation of European call option in M76 model via
#     Lewis (2001) Fourier-based approach: characteristic function.

#     Parameter definitions see function M76_value_call_INT. '''
#     omega = r - 0.5 * sigma ** 2 - lamb * (np.exp(mu + 0.5 * delta ** 2) - 1)
#     value = np.exp((1j * u * omega - 0.5 * u ** 2 * sigma ** 2 +
#             lamb * (np.exp(1j * u * mu - u ** 2 * delta ** 2 * 0.5) - 1)) * T)
#     return value


def strip_regularity_VG(beta, alpha):
    if alpha + beta >= 0 and beta - alpha <= -1:
        return True
    else:
        print('regularity conditions not satisfied')
        return False


def M76_characteristic_function(u, T, r, sigma, lamb, mu, delta, theta, V):
    ''' Valuation of European call option in M76 model via
    Lewis (2001) Fourier-based approach: characteristic function.

    Parameter definitions see function M76_value_call_INT. '''
    omega = r - 0.5 * sigma ** 2 - lamb * (np.exp(mu + 0.5 * delta ** 2) - 1)
    value = np.exp((1j * u * omega - 0.5 * u ** 2 * sigma ** 2 +
            lamb * (np.exp(1j * u * mu - u ** 2 * delta ** 2 * 0.5) - 1)) * T)
    

    value = np.exp(1j * u * omega) * (1 - 1j * u * theta * V + 0.5 * sigma ** 2 * V * u ** 2)

    alpha = np.sqrt((2 / V * sigma ** 2) + (theta ** 2 / sigma ** 4))
    beta = theta / sigma ** 2

    return value if strip_regularity_VG(beta, alpha) else 0
    

def strip_regularity_VG(beta, alpha):
    if alpha + beta >= 0 and beta - alpha <= -1:
        return True
    else:
        print('regularity conditions not satisfied')
        return False


    